Este trabajo fue hecho para aprender a hacer una red neuronal para resolver un problema de clasificación 

In [1]:
#importo las librerias, tesorflow ya está instalado en Colab
from __future__  import absolute_import,division,print_function,unicode_literals
import tensorflow as tf
import pandas as pd

In [2]:
#usamos IRIS DATASET

In [3]:
CSV_COLUMN_NAMES= ['Sepallength', 'Sepalwidth', 'Petallength', 'Petalwidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']
# definimos las listas de nombre de columnas y clases a determinar que usaremos luego

traigo el dataset de keras porque ya está separado en train y test

In [4]:
train_path= tf.keras.utils.get_file('Iris_training-csv', 'https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv')
test_path = tf.keras.utils.get_file("iris _test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train=pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)


573/573 [==============================] - 0s 0us/step


In [5]:
#Chequemos que hay en los datasets, dado que las columnas de categorías tienen nombres, hay que pasarlas a números
train.head()


,Sepallength,Sepalwidth,Petallength,Petalwidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [6]:
#separamos la columna label.
train_y = train.pop('Species')
test_y = test.pop( 'Species')
train. head() 


,Sepallength,Sepalwidth,Petallength,Petalwidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [7]:
train.shape # chequeo las dimensiones del dataset _ ojo, es pequeño para un entrenamiento de un modelo, pero es una forma simple de aprender a hacer redes, solo a los efectos del ejercicio

(120, 4)

In [8]:
train_y.head()

0    2
1    1
2    2
3    0
4    0
Name: Species, dtype: int64

In [9]:
def input_fn(features,labels,training=True,batch_size=256):
    #lo pasamos a un dataset. 
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    #Se usa Shuffle para entrenar mejor el modelo ya que tengo tan pocos datos, mezcla y repite (tiene un uso similar al cross_validation): 
    if training:
        dataset = dataset.shuffle(1000).repeat()
    return dataset.batch(batch_size)

In [10]:
#determino las columnas feature como input
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

[NumericColumn(key='Sepallength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Sepalwidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Petallength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Petalwidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [11]:
#construyo un modelo de red neuronal: DNN con 2 capas ocultas, la primera de 30 nodos y la segunda de 10 nodos.
classifier=tf.estimator.DNNClassifier(feature_columns=my_feature_columns, hidden_units=[30,10],n_classes=3)

In [12]:
#entrenamiento
classifier.train(input_fn=lambda:input_fn(train,train_y,training=True),steps=5000)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [13]:
#uso el modelo con el test y saco el accuracy
eval_result=classifier.evaluate(input_fn=lambda:input_fn(test,test_y, training=False))
print('\nTest set accuracy:{accuracy:0.3f}\n'.format(**eval_result))


Test set accuracy:0.933



In [14]:
#defino una funcion que permite ingresar los valores de las features cuando la corres y te da el % de probabilidad de que sea de cada una de las distintas clases)
def input_fn(features, batch_size=256):
# Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)
features = ['Sepallength', 'Sepalwidth', 'Petallength', 'Petalwidth']
predict={}

print("Please type numeric values as prompted.")
for feature in features:
    valid = True
    while valid:
        val = input(feature + ": ")
        if not val.isdigit(): valid = False
    
    predict[feature] = [float(val)]


predictions=classifier.predict(input_fn=lambda:input_fn(predict))
for pred_dict in predictions:
    print(pred_dict)
    class_id = pred_dict[ 'class_ids'][0]
    probability=pred_dict['probabilities'][class_id]

print('Prediction is "{}" ({:.1f}%)'.format(SPECIES[class_id], 100 * probability))

Please type numeric values as prompted.
Sepallength: 1.2
Sepalwidth: 2.1
Petallength: 3
Petallength: 1.2
Petalwidth: 2.1
{'logits': array([0.6524515, 1.788757 , 1.952617 ], dtype=float32), 'probabilities': array([0.12844981, 0.4001517 , 0.4713985 ], dtype=float32), 'class_ids': array([2]), 'classes': array([b'2'], dtype=object), 'all_class_ids': array([0, 1, 2], dtype=int32), 'all_classes': array([b'0', b'1', b'2'], dtype=object)}
Prediction is "Virginica" (47.1%)


In [15]:

'''
expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = {'Sepallength': [5.1, 5.9, 6.9],
'SepalWidth': [3.3, 3.0, 3.1],
'Petallength': [1.7, 4.2, 5.4],
'PetalWidth': [0.5, 1.5, 2.1],
}
'''